In [1]:
'''Carvana Image segmentation my solution with a pre-trained encoder'''

'Carvana Image segmentation my solution with a pre-trained encoder'

In [2]:
import torch 
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import os
from tqdm import  tqdm
import GPUtil as GPU
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
import cv2
from torchvision.transforms import *


In [3]:
train_mask=pd.read_csv('train_masks.csv')
metadata=pd.read_csv('metadata.csv')
submission=pd.read_csv('sample_submission.csv')

In [4]:
train_images= os.listdir('train_hq')
test_images= os.listdir('test_hq')
train_masks= os.listdir('train_masks')
train_path= [os.path.join('train_hq',i) for i in os.listdir('train_hq')]
test_path=[os.path.join('test_hq',i) for i in os.listdir('test_hq')]
mask_path=[os.path.join('train_masks',i) for i in os.listdir('train_masks')]

In [5]:
print(len(test_images)/16)

6254.0


There are 318 training images, and 6254 test images. No of training images>test images so that might create a problem. Valdimir approaches this by pseudo labelling, see what it is later on.

In [6]:
from PIL import Image
def plot_image_and_mask(train_images):
    #plot the train_images
    figure,axis=plt.subplots(len(train_images),2,figsize=(2*10,10*len(train_images)))
    for i in range(len(train_images)):
        id=train_images[i]
        image_path=os.path.join('train',id)
        mask_path=os.path.join('train_masks',id)[:-4]+'_mask.gif'
        print(np.array(Image.open(image_path)).shape)
        axis[i][0].imshow(Image.open(image_path))
        axis[i][1].imshow(Image.open(mask_path))

In [7]:
# plot_image_and_mask(train_images[50:70])

shutil craetes symbolic link to the directories where the files are copied. When we create folds, the data does not disappear.

In [8]:
# #test code to create the folds 
# from pathlib import Path
# import shutil
# local_data_path=Path('.').absolute()/'folds'
# local_data_path.mkdir(exist_ok=True)
# train_path= Path('.').absolute()/'train_hq/'
# #train_path= Path('.').absolute()/'train/'
# mask_path=Path('.').absolute()/'train_masks'
# train_file_list=train_images
# folds=pd.read_csv('folds_csv.csv')
# num_folds=folds.fold.nunique()
# #the cars are at sixteen angles 
# angles=['0'+ str(i) for i in range (1,10)]+ [str(i) for i in range(10,17)]
# #now create the folder containing the folds 
# for fold in range(num_folds):
#     #create the folder correspoonding to number of folds
#     (local_data_path / str(fold) / 'train' / 'images').mkdir(exist_ok=True, parents=True)
#     (local_data_path / str(fold) / 'train' / 'masks').mkdir(exist_ok=True, parents=True)

#     (local_data_path / str(fold) / 'val' / 'images').mkdir(exist_ok=True, parents=True)
#     (local_data_path / str(fold) / 'val' / 'masks').mkdir(exist_ok=True, parents=True)

# for i in tqdm(folds.index):
#     car_id=folds.loc[i,'id']
#     fold=folds.loc[i,'fold']
#     #copy the 16 images to fold directory 
#     #this is considered as a val this time 
#     for angle in angles:
#         old_image_path = train_path / (car_id + '_' + angle + '.jpg')

#         new_image_path = local_data_path / str(fold) / 'val' / 'images' / (car_id + '_' + angle + '.jpg')
#         shutil.copy(str(old_image_path), str(new_image_path))

#         old_mask_path = mask_path / (car_id + '_' + angle + '_mask.gif')
#         new_mask_path = local_data_path / str(fold) / 'val' / 'masks' / (car_id + '_' + angle + '_mask.gif')
#         shutil.copy(str(old_mask_path), str(new_mask_path))
        
#     #for all the other folds this will be considered as a  training image
#     for t_fold in range(num_folds):
#             if t_fold == fold:
#                 continue

#             for angle in angles:
#                 old_image_path = train_path / (car_id + '_' + angle + '.jpg')

#                 new_image_path = local_data_path / str(t_fold) / 'train' / 'images' / (car_id + '_' + angle + '.jpg')
#                 shutil.copy(str(old_image_path), str(new_image_path))

#                 old_mask_path = mask_path / (car_id + '_' + angle + '_mask.gif')
#                 new_mask_path = local_data_path / str(t_fold) / 'train' / 'masks' / (car_id + '_' + angle + '_mask.gif')
#                 shutil.copy(str(old_mask_path), str(new_mask_path))


We take care of the modelling now. The original implementation used unet 11 with a pre-trained vgg-11 encoder, which is what we will use.

In [9]:
#modelling
from torch import nn 
from torch.nn import functional as F
import torch 
from torchvision import models

Vladimir uses the Unet architecture for semantic segmentation. The modifications to the net are taurusunet where a pre-trained encoder is used for the segmentation. Let's try to do the modelling here and try to understand the input dimensions.

In [10]:
#handle multiple gpus in the jupyter notebook 
device=torch.device("cuda:1" if (torch.cuda.is_available()) else "cpu")

if device.type=='cuda':
    gpu_id=[]
    GPUs = GPU.getGPUs()
    for i in range(len(GPUs)):
        gpu = GPUs[i]
        if gpu.memoryFree>5000:
            gpu_id.append(i)

In [11]:
def conv3x3(in_, out):
    return nn.Conv2d(in_, out, 3, padding=1)


def concat(xs):
    return torch.cat(xs, 1)


class Conv3BN(nn.Module):
    def __init__(self, in_: int, out: int, bn=False):
        super().__init__()
        self.conv = conv3x3(in_, out)
        self.bn = nn.BatchNorm2d(out) if bn else None
        self.activation = nn.SELU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        x = self.activation(x)
        return x

    
class DecoderBlock(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels):
        super().__init__()

        self.block = nn.Sequential(
            ConvRelu(in_channels, middle_channels),
            nn.ConvTranspose2d(middle_channels, out_channels, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.block(x)
    

class ConvRelu(nn.Module):
    def __init__(self, in_: int, out: int):
        super().__init__()
        self.conv = conv3x3(in_, out)
        self.activation = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.activation(x)
        return x


In [12]:

class UNet11(nn.Module):
    def __init__(self, num_classes=1, num_filters=32):
        super().__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.encoder = models.vgg11(pretrained=True).features
        self.relu = self.encoder[1]
        self.conv1 = self.encoder[0]
        self.conv2 = self.encoder[3]
        self.conv3s = self.encoder[6]
        self.conv3 = self.encoder[8]
        self.conv4s = self.encoder[11]
        self.conv4 = self.encoder[13]
        self.conv5s = self.encoder[16]
        self.conv5 = self.encoder[18]

        self.center = DecoderBlock(num_filters * 8 * 2, num_filters * 8 * 2, num_filters * 8)
        self.dec5 = DecoderBlock(num_filters * (16 + 8), num_filters * 8 * 2, num_filters * 8)
        self.dec4 = DecoderBlock(num_filters * (16 + 8), num_filters * 8 * 2, num_filters * 4)
        self.dec3 = DecoderBlock(num_filters * (8 + 4), num_filters * 4 * 2, num_filters * 2)
        self.dec2 = DecoderBlock(num_filters * (4 + 2), num_filters * 2 * 2, num_filters)
        self.dec1 = ConvRelu(num_filters * (2 + 1), num_filters)

        self.final = nn.Conv2d(num_filters, num_classes, kernel_size=1)

    def forward(self, x):
        conv1 = self.relu(self.conv1(x))
        conv2 = self.relu(self.conv2(self.pool(conv1)))
        conv3s = self.relu(self.conv3s(self.pool(conv2)))
        conv3 = self.relu(self.conv3(conv3s))
        conv4s = self.relu(self.conv4s(self.pool(conv3)))
        conv4 = self.relu(self.conv4(conv4s))
        conv5s = self.relu(self.conv5s(self.pool(conv4)))
        conv5 = self.relu(self.conv5(conv5s))

        center = self.center(self.pool(conv5))

        dec5 = self.dec5(torch.cat([center, conv5], 1))
        dec4 = self.dec4(torch.cat([dec5, conv4], 1))
        dec3 = self.dec3(torch.cat([dec4, conv3], 1))
        dec2 = self.dec2(torch.cat([dec3, conv2], 1))
        dec1 = self.dec1(torch.cat([dec2, conv1], 1))
        return F.sigmoid(self.final(dec1))

    
#writing the dice based loss weights for the neural network 

class Loss:
    #init script for the loss
    def __init__(self, dice_weight=1):
        self.nll_loss = nn.BCELoss()
        self.dice_weight = dice_weight

    #call called if parameters are given to the network 
    def __call__(self, outputs, targets):
        loss = self.nll_loss(outputs, targets)
        if self.dice_weight:
            eps = 1e-15
            dice_target = (targets == 1).float()
            dice_output = outputs
            intersection = (dice_output * dice_target).sum()
            union = dice_output.sum() + dice_target.sum() + eps

            loss -= torch.log(2 * intersection / union)

        return loss


In [13]:
from pathlib import Path
import shutil
#create a dataloader for the carvana dataset
train_images= os.listdir('train_hq')
test_images= os.listdir('test_hq')
train_masks= os.listdir('train_masks')
train_path= [os.path.join('train_hq',i) for i in os.listdir('train_hq')]
test_path=[os.path.join('test_hq',i) for i in os.listdir('test_hq')]
mask_path=[os.path.join('train_masks',i) for i in os.listdir('train_masks')]

'''returns the padded image'''
#writing the image loader function 
def load_image(path):
    img=cv2.imread(path)
    img = cv2.copyMakeBorder(img, 0, 0, 1, 1, cv2.BORDER_REFLECT_101)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #img=cv2.resize(img,(img.shape[1]//2,img.shape[0]//2))
    return img.astype(np.uint8)
'''returns the mask'''
def load_mask(path):
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            if '.gif' in str(path):
                img = (np.asarray(img) > 0)
            else:
                img = (np.asarray(img) > 255 * 0.5)
            img = cv2.copyMakeBorder(img.astype(np.uint8), 0, 0, 1, 1, cv2.BORDER_REFLECT_101)
           # img=cv2.resize(img,(img.shape[1]//2,img.shape[0]//2))
            return img.astype(np.float32)
        
img_transform = Compose([
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
        
'''since we are feeding the images to pretrained vgg11, we need to normalize the weights
https://pytorch.org/docs/stable/torchvision/models.html
make a transformation pipeline for input images in the carvana dataset'''
#our dataloader should also handle the image augmentation 
#inherit the Dataset

class CarvanaDataset(Dataset):
    def __init__(self, root: Path, to_augment=False):
        # TODO This potentially may lead to bug.
        self.image_paths = sorted(root.joinpath('images').glob('*.jpg'))
        self.mask_paths = sorted(root.joinpath('masks').glob('*'))
        self.to_augment = to_augment

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img=load_image(str(self.image_paths[idx]))
        mask = load_mask(str(self.mask_paths[idx]))
#

        return img_transform(img), torch.from_numpy(np.expand_dims(mask, 0))


In [14]:
local_data_path=Path('.').absolute()/'folds'
a=CarvanaDataset(local_data_path/str(2)/'train')
fold_path=sorted((local_data_path/str(2)/'train').joinpath('images').glob('*.jpg'))
fold_path[0]
a[0]

#load_image('/data2/6666/rajat.modi/practice/Carvana/carvana/CarvanaSegmentation/folds/2/train/images/00087a6bd4dc_01.jpg')

(tensor([[[2.0434, 2.0434, 2.0434,  ..., 2.0434, 2.0434, 2.0434],
          [2.0434, 2.0434, 2.0434,  ..., 2.0434, 2.0434, 2.0434],
          [2.0434, 2.0434, 2.0434,  ..., 2.0434, 2.0434, 2.0434],
          ...,
          [0.9303, 0.9303, 0.9303,  ..., 0.9132, 0.9132, 0.9132],
          [0.9303, 0.9303, 0.9303,  ..., 0.9132, 0.8961, 0.9132],
          [0.9132, 0.9132, 0.9132,  ..., 0.9132, 0.9132, 0.9132]],
 
         [[2.2185, 2.2185, 2.2185,  ..., 2.2185, 2.2185, 2.2185],
          [2.2185, 2.2185, 2.2185,  ..., 2.2185, 2.2185, 2.2185],
          [2.2185, 2.2185, 2.2185,  ..., 2.2185, 2.2185, 2.2185],
          ...,
          [1.1506, 1.1506, 1.1506,  ..., 1.0980, 1.0980, 1.0980],
          [1.1506, 1.1506, 1.1506,  ..., 1.0980, 1.0805, 1.0980],
          [1.1331, 1.1331, 1.1331,  ..., 1.0980, 1.0980, 1.0980]],
 
         [[2.3960, 2.3960, 2.3960,  ..., 2.4308, 2.4308, 2.4308],
          [2.3960, 2.3960, 2.3960,  ..., 2.4308, 2.4308, 2.4308],
          [2.3960, 2.3960, 2.3960,  ...,

In [15]:
def make_loader(ds_root: Path, to_augment=False, shuffle=False):
    return DataLoader(
        dataset=CarvanaDataset(ds_root, to_augment=to_augment),
        shuffle=shuffle,
        num_workers=1,
        batch_size=4,
        pin_memory=True
    )

gpu_id

[0, 1, 2]

In [16]:
######################VALIDATION FUNCTION#################
batch_size=4
def get_dice_score(y_true,y_pred):
    epsilon=1e-15
    intersection= (y_true*y_pred).sum()
    union=y_true.sum()+y_pred.sum()+epsilon
    return 2*(intersection/union)

def validation(model,criterion,valid_loader):
    model.eval()
    losses=[]
    dice=[]
    tq=tqdm(total=len(valid_loader)*batch_size)
    for inputs,target in valid_loader:
        target=target.to(device)
        output=model(inputs).to(device)
        loss=criterion(output,target)
        current_batch_size=inputs.size(0)
        losses.append(loss.item())
        tq.update(current_batch_size)
        dice.append(get_dice_score(target,(output>0.5).float()).item())
    
    #take a mean of the loss and dice score here
    tq.close()
    valid_loss=np.mean(losses)
    valid_dice=np.mean(dice)
    print('Valid loss: {:.5f}, dice: {:.5f}'.format(valid_loss, valid_dice))
    metrics = {'valid_loss': valid_loss, 'dice_loss': valid_dice}
    return metrics

# dice=validation(model,criterion,val_loader)
# print(dice)

In [17]:
#function for cyclic learning rate 
def cyclic_lr(epoch, init_lr=1e-4, num_epochs_per_cycle=5, cycle_epochs_decay=2, lr_decay_factor=0.5):
    epoch_in_cycle = epoch % num_epochs_per_cycle
    lr = init_lr * (lr_decay_factor ** (epoch_in_cycle // cycle_epochs_decay))
    return lr
 

In [ ]:
#for each fold, load its train and val sets
for fold_no in range(4,5):
    
    print("initializing the model",)
    #create the model 
    model=UNet11().to(device)
    criterion=Loss() 
    model=nn.DataParallel(model,device_ids=[1,2])
    print("fold is",fold_no) 
    train_loader=make_loader(local_data_path/str(fold_no)/'train')
    val_loader=make_loader(local_data_path/str(fold_no)/'val')

    #####################SAVE MODEL PATH BEGINS#################
    model_save_path=Path('.').absolute()/'modelweights'
    model_save_path.mkdir(exist_ok=True, parents=True)
    best_model_save_path=Path('.').absolute()/'bestmodelweights'
    best_model_save_path.mkdir(exist_ok=True, parents=True)

    model_fold='model_{fold}.pt'.format(fold=fold_no)
    save =lambda ep:torch.save({
            'model': model.state_dict(),
            'epoch': ep,
            'best_valid_loss': best_valid_loss
        }, str(model_save_path/str(model_fold)))
    ###################MODEL SAVE ENDS#######################
    #################TRAINING FUNCTION########################
    print('hello')
    batch_size=4
    n_epochs=20
    lr= 0.0001
    report_loss_threshold=10
    optimizer= Adam(model.parameters(),lr=lr)
    #criterion=loss
    best_valid_loss = float('inf')
    valid_losses=[]
    for epoch in range(1,n_epochs+1):
        lr=cyclic_lr(epoch)
        print("learning rate is ",lr)
        optimizer=Adam(model.parameters(),lr=lr)
        model.train()
        #define the progressbar here
        tq=tqdm(total=len(train_loader)*batch_size)
        tq.set_description('Epoch {}, lr {}'.format(epoch, lr))
        losses=[]
        #main training loop comes here

        for i,(inputs,target) in enumerate(train_loader):
            #inputs=inputs.to(device)
            target=target.to(device)
            #print(inputs.size)
            output=model(inputs).to(device)
            #print(output.size())
            loss=criterion(output,target)
            optimizer.zero_grad()
            current_batch_size=inputs.size(0)
            tq.update(current_batch_size)
            #print(loss.item())
            losses.append(loss.item())
            mean_loss=np.mean(losses[-report_loss_threshold:])
            #set the postfix of the progress bar 
            tq.set_postfix(loss='{:.5f}'.format(mean_loss))
            (current_batch_size*loss).backward()
            optimizer.step()

        #close the progress bar 
        tq.close()
        print("saving model epoch",epoch)
        save(epoch)
        valid_metrics=validation(model,criterion,val_loader)
        valid_loss=valid_metrics['valid_loss']
        valid_losses.append(valid_loss)
        #check if the val loss is less than the best loss
        if valid_loss<best_valid_loss:
            print("better_val_loss model found",valid_loss)
            best_valid_loss=valid_loss
            #copy the model to the best_model_loss directory
            shutil.copy(str(model_save_path/str(model_fold)),str(best_model_save_path/str(model_fold)))  

initializing the model


Epoch 1, lr 0.0001:   0%|          | 0/4080 [00:00<?, ?it/s]

fold is 4
hello
learning rate is  0.0001


/data2/6666/rajat.modi/anaconda3/envs/gpuenv/lib/python3.7/site-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
Epoch 1, lr 0.0001: 100%|██████████| 4080/4080 [15:31<00:00,  4.48it/s, loss=0.01346]


saving model epoch 1


...

Epoch 17, lr 5e-05: 100%|██████████| 4080/4080 [15:13<00:00,  4.47it/s, loss=0.00566]


saving model epoch 17


Epoch 18, lr 5e-05:   0%|          | 0/4080 [00:00<?, ?it/s]

Valid loss: 0.00802, dice: 0.99676
learning rate is  5e-05


Epoch 18, lr 5e-05:  53%|█████▎    | 2160/4080 [08:03<07:13,  4.42it/s, loss=0.00745]

In [ ]:
#######################PREDICTION LOOP##############################
#writing the code for prediction
#rewrite the dataloader for test dataset, since no masks are present 
#we will predict the masks for val and test_hq for each fold

img_transform = Compose([
    ToTensor(),dataset=PredictionDatasetPure(from_path),
            shuffle=False,
            batch_size=batch_size,
            num_workers=1,
            pin_memory=True
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
def load_image(path):
    img=cv2.imread(path)
    img = cv2.copyMakeBorder(img, 0, 0, 1, 1, cv2.BORDER_REFLECT_101)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #img=cv2.resize(img,(img.shape[1]//2,img.shape[0]//2))
    return img.astype(np.uint8)
#dataset to load the prediction images
class PredictionDatasetPure:
    def __init__(self, paths):
        self.paths = paths

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx % len(self.paths)]
        print(path)
        image = load_image(str(path))
        #path.stem returns the path 
        return img_transform(image),path.stem
    
###################WE PREDICT HERE###################################################    
#function to take a from path, to path & to paste the predictions

def predict(model, from_paths, batch_size: int, to_path):
    loader = DataLoader(
        dataset=PredictionDatasetPure(from_paths),
        shuffle=False,
        batch_size=batch_size,
        num_workers=1,
        pin_memory=True
    )

    for batch_num, (inputs) in enumerate(tqdm(loader, desc='Predict')):
        inputs = utils.variable(inputs, volatile=True)
        outputs = model(inputs)
        mask = (outputs.data.cpu().numpy() * 255).astype(np.uint8)

        for i, image_name in enumerate(stems):
            cv2.imwrite(str(to_path / (stems[i] +img_transform '.png')), mask[i, 0, :, 1:-1])

batch_size=4
           
fold_no=4
local_data_path=Path('.').absolute()restore the model from here
prediction_path=(local_data_path/'predictions').mkdir(exist_ok=True, parents=True)
val_images=sorted(list((local_data_path/'folds'/str(fold_no)/'val'/'images').glob('*.jpg')))
len_val=len(val_images)
test_images=sorted(list((local_data_path/'test_hq').glob('*.jpg')))
len_test=len(test_images)



In [ ]:
a=PredictionDatasetPure(test_images)
_,b=a[0]
b

In [ ]:
load_image(path)